# Integration Test

Calls AlphaPept for various files / settings and see if they work / how they perform.

In [1]:
from time import time
%load_ext autoreload
%autoreload 2

## Logger

In [2]:
import sys
import logging
# Create logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create STDERR handler
handler = logging.StreamHandler(sys.stderr)
# ch.setLevel(logging.DEBUG)

# Create formatter and add it to the handler
formatter = logging.Formatter('%(asctime)s %(levelname)-s - %(message)s', "%Y-%m-%d %H:%M:%S")
handler.setFormatter(formatter)

# Set STDERR handler as the only handler 
logger.handlers = [handler]

## Integration Tests

Read folder `integration_test` and see what files we have.

In [3]:
import os

test_dir = './integration_test/'

files = os.listdir('./integration_test/')

raw_files = None
fasta_file = None

for file in files:
    base, ext = os.path.splitext(file)
    
    if ext == '.fasta':
        fasta_file = os.path.abspath(os.path.join(test_dir, file))
        print('Found fasta file: {}'.format(fasta_file))
    elif ext == '.raw':
        if not raw_files:
            raw_files = []
            raw_files.append(os.path.abspath(os.path.join(test_dir, file)))
        else:
            raw_files.append(os.path.abspath(os.path.join(test_dir, file)))
        print('Found thermo file: {}'.format(file))
    elif ext == '.d':
        if not raw_files:
            raw_files = []
            raw_files.append(os.path.abspath(os.path.join(test_dir, file)))
        else:
            raw_files.append(os.path.abspath(os.path.join(test_dir, file)))
        print('Found bruker file: {}'.format(file))
    else:
        print('File not recognized: {}'.format(file))
        
        
#Todo: Later combine files that might belong together (i.e. in folder, or of same type)

File not recognized: 20190402_QX1_SeVW_MA_HeLa_500ng_LC11.npz
Found thermo file: 20190402_QX1_SeVW_MA_HeLa_500ng_LC11.raw
File not recognized: 20190402_QX1_SeVW_MA_HeLa_500ng_LC11_ap.csv
File not recognized: 20190402_QX1_SeVW_MA_HeLa_500ng_LC11_ap.yaml
Found bruker file: 20191209_TIMS05_QC_ADB_AfterSRIGUpgrade_200ng_03_A1_1_1428.d
File not recognized: 20191209_TIMS05_QC_ADB_AfterSRIGUpgrade_200ng_03_A1_1_1428.npz
File not recognized: 20191209_TIMS05_QC_ADB_AfterSRIGUpgrade_200ng_03_A1_1_1428_ap.csv
File not recognized: 20191209_TIMS05_QC_ADB_AfterSRIGUpgrade_200ng_03_A1_1_1428_ap.yaml
Found fasta file: F:\projects\alphapept\integration_test\uniprot_Human_reviewed_March_2019_manual.fasta
File not recognized: uniprot_Human_reviewed_March_2019_manual.npz


## Load settings and create default settings

In [4]:
from alphapept.settings import load_settings

default_settings = load_settings('default_settings.yaml')

def return_bool_settings(settings):
    """
    Extract all settings that are of boolean type
    """

    boolean_settings = []

    for category in settings.keys():

        temp_settings = {}

        for key in settings[category].keys():

            setting = settings[category][key]
            if type(setting) == bool:  
                boolean_settings.append((category, key, setting))
                

    return boolean_settings

# Deactivate maxlfq and match between runs in the default case as they are not implemented
# Also deactivete library creation
# Otherwise: Activate all functions and deactivate to test

default_settings['quantification']['max_lfq'] = False
default_settings['misc']['match_between_runs'] = False
default_settings['general']['create_library'] = False

#Fill with paths from above 
default_settings['fasta']['fasta_path'] = 'F:/projects/alphapept/integration_test/uniprot_Human_reviewed_March_2019_manual.fasta'
default_settings['fasta']['contaminants_path'] = 'F:/projects/alphapept/contaminants.fasta'

boolean_settings = return_bool_settings(default_settings)

## IO Tests

In [5]:
import copy 

test_cases = []

#File conversion tests: Convert to AlphaPept NPZ container, Create library

for file in raw_files:
    for check in [True, False]:
        
        test_case = copy.deepcopy(default_settings)
        
        test_case['raw']['raw_path'] = file
        
        base, ext = os.path.splitext(test_case["raw"]["raw_path"])
    
        if ext == '.raw':
            datatype='thermo'
        elif ext == '.d':
            datatype='bruker'
        else:
            raise NotImplementedError('File extension {} not understood.'.format(ext))

        #Adjust boundaries for thermo file

        if datatype =='bruker':
            test_case['search']['m_offset'] = 50
            test_case['search']['m_tol'] = 50
        
        if check:
            test_case['raw']['convert_raw'] = True
            test_case['general']['create_library'] = True
        
        else:
            raw_path_npz = os.path.splitext(file)[0] + '.npz'
            test_case['raw']['raw_path_npz'] = raw_path_npz
            test_case['fasta']['library_path'] = os.path.splitext(test_case['fasta']['fasta_path'])[0] + '.npz'
            test_case['general']['create_library'] = False
            test_case['raw']['convert_raw'] = False
               
        test_cases.append(test_case)
        
print('IO test with {} runs created.'.format(len(test_cases)))       

IO test with 4 runs created.


## Module Integration tests

In [6]:
for file in raw_files:
    
    for flags in boolean_settings:
        cat, key, val = flags
        
        if key not in ['convert_raw','create_library','parallel','ppm']:
            
            test_case = copy.deepcopy(default_settings)
            
            test_case['raw']['raw_path'] = file
            test_case['raw']['convert_raw'] = False
            test_case['raw']['raw_path_npz'] = os.path.splitext(file)[0] + '.npz'
            test_case['fasta']['library_path'] = os.path.splitext(test_case['fasta']['fasta_path'])[0] + '.npz'
            test_case[cat][key] = not val 

            base, ext = os.path.splitext(test_case["raw"]["raw_path"])
    
            if ext == '.raw':
                datatype='thermo'
            elif ext == '.d':
                datatype='bruker'
            else:
                raise NotImplementedError('File extension {} not understood.'.format(ext))
            
            #Adjust boundaries for thermo file
            
            if datatype =='bruker':
                test_case['search']['m_offset'] = 50
                test_case['search']['m_tol'] = 50

            test_cases.append(test_case)
            
print('Integration test with {} runs created.'.format(len(test_cases)))

Integration test with 12 runs created.


## Check wheter all boolean flags are considered

In [7]:
import numpy as np

test_check = []

for test_case in test_cases:
    
    test_check.append(tuple([_[2] for _ in return_bool_settings(test_case)]))
    
import pandas as pd

columns = [_[1] for _ in return_bool_settings(test_case)]
#columns.extend(['n_proteins', 'n_sequences', 'n_sequence_charges', 'time_minutes', 'error'])
test_summary = pd.DataFrame(test_check, columns = columns)

# Check that all options are true at least once
test_summary_check = np.sum(test_summary)
not_tested = test_summary.columns[(test_summary_check==0)]

if len(not_tested) > 0:
    print('Error. The following options are never tested {}.'.format([_ for _ in not_tested]))
    raise
else:
    print('Test coverage looks good.')

Test coverage looks good.


In [9]:
from time import time
import numpy as np
from alphapept.runner import alpha_runner

def run_test(settings):
    """
    Run a test with certain settings
    """

    
    def dummy(input):
        pass
    
    with tqdm(total=100, desc='Current Progress', unit='%') as current_pbar:
        with tqdm(total=100, desc='Overall Progress', unit='%') as overall_pbar:

            def current_progress(current):
                to_update = np.round(current*100-current_pbar.n, 2)
                current_pbar.update(to_update)

            def overall_progress(current):
                to_update = np.round(current-overall_pbar.n, 2)
                overall_pbar.update(to_update)
                
                
            start = time()
            try:
                df, features = alpha_runner(settings, overall_progress = overall_progress, current_progress = current_progress, CURRENT_TASK = dummy)
                error = False
            except Exception as e:
                print('Experienced the following exception:')
                print('-'*100)
                print(e)
                print('-'*100)
                df = None
                error = True
                
            end = time()
            
            
    if df is not None:
        n_proteins = len(set(df['protein']))
        n_sequences = len(set(df['sequence']))
        n_sequence_charges = len(set(df['sequence'] + df['charge'].astype('int').astype('str')))
    else:
        n_proteins = np.nan
        n_sequences = np.nan
        n_sequence_charges = np.nan      
    
    time_minutes = (end-start)/60
    
    return (n_proteins, n_sequences, n_sequence_charges, time_minutes, error)

In [10]:
if False:
    import pandas as pd
    from tqdm.notebook import tqdm as tqdm

    report = []
    start = time()
    for test_case in tqdm(test_cases):

        report.append(tuple([_[2] for _ in return_bool_settings(test_case)]) + run_test(test_case))

    end = time()

    import pandas as pd

    print('Integration test complete. Time elapsed {:.2f} hours'.format((end-start)/60/60))

    columns = [_[1] for _ in return_bool_settings(test_case)]
    columns.extend(['n_proteins', 'n_sequences', 'n_sequence_charges', 'time_minutes', 'error'])

    print('Complete')
    pd.DataFrame(report, columns = columns)

2020-05-07 18:55:53 INFO - Checking raw path F:\projects\alphapept\integration_test\20190402_QX1_SeVW_MA_HeLa_500ng_LC11.raw.
2020-05-07 18:55:53 INFO - Found raw but no NPZ present. Performing conversion step.
2020-05-07 18:55:53 INFO - Raw path okay.
2020-05-07 18:55:53 INFO - Checking library path.
2020-05-07 18:55:53 INFO - No NPZ for library present. Creating library from FASTA.
2020-05-07 18:55:53 INFO - Library path okay.
2020-05-07 18:57:20 INFO - Digested 20,664 proteins and generated 5,289,587 peptides
2020-05-07 18:59:30 INFO - Generated 5,289,587 spectra
2020-05-07 19:00:08 INFO - Database saved to F:/projects/alphapept/integration_test/uniprot_Human_reviewed_March_2019_manual.npz. Filesize 3.28 Gb
2020-05-07 19:00:08 INFO - Imported existing <module 'comtypes.gen' from 'C:\\ProgramData\\Anaconda3\\envs\\alphap\\lib\\site-packages\\comtypes\\gen\\__init__.py'>
2020-05-07 19:00:08 INFO - Using writeable comtypes cache directory: 'C:\ProgramData\Anaconda3\envs\alphap\lib\site

2020-05-07 19:21:24 INFO - Checking raw path F:\projects\alphapept\integration_test\20190402_QX1_SeVW_MA_HeLa_500ng_LC11.raw.
2020-05-07 19:21:24 INFO - NPZ for raw file present. Skipping conversion step.
2020-05-07 19:21:24 INFO - Raw path okay.
2020-05-07 19:21:24 INFO - Checking library path.
2020-05-07 19:21:24 INFO - NPZ for library file present.
2020-05-07 19:21:24 INFO - Library path okay.
2020-05-07 19:21:24 INFO - Finding Features
2020-05-07 19:21:24 INFO - Fature Finding for Thermo
2020-05-07 19:21:45 INFO - Loaded 13,230 centroids.
2020-05-07 19:27:18 INFO - A total of 992,051 hills extracted. Average hill length 16.04
2020-05-07 19:28:48 INFO - Split 992,051 hills into 1,239,960 hills
2020-05-07 19:29:16 INFO - Filtered 1,239,960 hills. Remaining 1,221,467 hills
2020-05-07 19:33:12 INFO - Extracting hill stats complete
2020-05-07 19:34:40 INFO - Found 200044 pre isotope patterns.
2020-05-07 19:36:30 INFO - Extracted 218781 isotope patterns.
2020-05-07 19:37:14 INFO - Report

2020-05-07 19:40:18 INFO - Checking raw path F:\projects\alphapept\integration_test\20191209_TIMS05_QC_ADB_AfterSRIGUpgrade_200ng_03_A1_1_1428.d.
2020-05-07 19:40:18 INFO - Found raw but no NPZ present. Performing conversion step.
2020-05-07 19:40:18 INFO - Raw path okay.
2020-05-07 19:40:18 INFO - Checking library path.
2020-05-07 19:40:18 INFO - No NPZ for library present. Creating library from FASTA.
2020-05-07 19:40:18 INFO - Library path okay.
2020-05-07 19:41:35 INFO - Digested 20,664 proteins and generated 5,289,587 peptides
2020-05-07 19:43:45 INFO - Generated 5,289,587 spectra
2020-05-07 19:44:23 INFO - Database saved to F:/projects/alphapept/integration_test/uniprot_Human_reviewed_March_2019_manual.npz. Filesize 3.28 Gb
2020-05-07 20:13:36 INFO - Raw file(s) saved to ['F:\\projects\\alphapept\\integration_test\\20191209_TIMS05_QC_ADB_AfterSRIGUpgrade_200ng_03_A1_1_1428.npz']
2020-05-07 20:13:36 INFO - Finding Features
2020-05-07 20:13:36 INFO - Fature Finding for Bruker
2020-

2020-05-07 20:23:43 INFO - Checking raw path F:\projects\alphapept\integration_test\20191209_TIMS05_QC_ADB_AfterSRIGUpgrade_200ng_03_A1_1_1428.d.
2020-05-07 20:23:43 INFO - NPZ for raw file present. Skipping conversion step.
2020-05-07 20:23:43 INFO - Raw path okay.
2020-05-07 20:23:43 INFO - Checking library path.
2020-05-07 20:23:43 INFO - NPZ for library file present.
2020-05-07 20:23:43 INFO - Library path okay.
2020-05-07 20:23:43 INFO - Finding Features
2020-05-07 20:23:43 INFO - Fature Finding for Bruker
2020-05-07 20:23:45 INFO - Extracted 261,930 features.
2020-05-07 20:26:49 INFO - First search complete. Compared 356,163,285 spectra and found 1,947,349 psms.
2020-05-07 20:28:43 INFO - Extracted columns for 1,947,349 psms.
2020-05-07 20:28:50 INFO - Scoring complete. For 0.01 FDR found 137,748 targets and 1,377 decoys.
2020-05-07 20:28:50 INFO - Precursor Offset (PPM) is -16.85 (mean), 7.23 (std)
2020-05-07 20:29:55 INFO - Calibration complete. Precursor Offset (PPM) is -0.02 

2020-05-07 20:32:03 INFO - Checking raw path F:\projects\alphapept\integration_test\20190402_QX1_SeVW_MA_HeLa_500ng_LC11.raw.
2020-05-07 20:32:03 INFO - NPZ for raw file present. Skipping conversion step.
2020-05-07 20:32:03 INFO - Raw path okay.
2020-05-07 20:32:03 INFO - Checking library path.
2020-05-07 20:32:03 INFO - NPZ for library file present.
2020-05-07 20:32:03 INFO - Library path okay.
2020-05-07 20:32:03 INFO - Using Features from raw data
2020-05-07 20:32:21 INFO - First search complete. Compared 30,099,867 spectra and found 175,590 psms.
2020-05-07 20:32:38 INFO - Extracted columns for 175,590 psms.
2020-05-07 20:32:38 INFO - Scoring complete. For 0.01 FDR found 62,370 targets and 623 decoys.
2020-05-07 20:32:38 INFO - Precursor Offset (PPM) is -0.24 (mean), 2.48 (std)
2020-05-07 20:33:38 INFO - Calibration complete. Precursor Offset (PPM) is 0.00 (mean), 0.84 (std)
2020-05-07 20:33:38 INFO - Adjusting search bound to 2.52 ppm.
2020-05-07 20:33:45 INFO - Second search com

...

Experienced the following exception:
----------------------------------------------------------------------------------------------------
Match beween runs not implemented yet.
----------------------------------------------------------------------------------------------------




2020-05-07 21:47:06 INFO - Checking raw path F:\projects\alphapept\integration_test\20191209_TIMS05_QC_ADB_AfterSRIGUpgrade_200ng_03_A1_1_1428.d.
2020-05-07 21:47:06 INFO - NPZ for raw file present. Skipping conversion step.
2020-05-07 21:47:06 INFO - Raw path okay.
2020-05-07 21:47:06 INFO - Checking library path.
2020-05-07 21:47:06 INFO - NPZ for library file present.
2020-05-07 21:47:06 INFO - Library path okay.
2020-05-07 21:47:06 INFO - Finding Features
2020-05-07 21:47:06 INFO - Fature Finding for Bruker
2020-05-07 21:47:08 INFO - Extracted 261,930 features.
2020-05-07 21:50:10 INFO - First search complete. Compared 356,163,285 spectra and found 1,947,349 psms.
2020-05-07 21:52:04 INFO - Extracted columns for 1,947,349 psms.
2020-05-07 21:52:10 INFO - Scoring complete. For 0.01 FDR found 137,748 targets and 1,377 decoys.
2020-05-07 21:52:10 INFO - Precursor Offset (PPM) is -16.85 (mean), 7.23 (std)
2020-05-07 21:53:15 INFO - Calibration complete. Precursor Offset (PPM) is -0.02 

Experienced the following exception:
----------------------------------------------------------------------------------------------------
MaxLFQ not implemented yet.
----------------------------------------------------------------------------------------------------




2020-05-07 21:55:21 INFO - Checking raw path F:\projects\alphapept\integration_test\20191209_TIMS05_QC_ADB_AfterSRIGUpgrade_200ng_03_A1_1_1428.d.
2020-05-07 21:55:21 INFO - NPZ for raw file present. Skipping conversion step.
2020-05-07 21:55:21 INFO - Raw path okay.
2020-05-07 21:55:21 INFO - Checking library path.
2020-05-07 21:55:21 INFO - NPZ for library file present.
2020-05-07 21:55:21 INFO - Library path okay.
2020-05-07 21:55:21 INFO - Finding Features
2020-05-07 21:55:21 INFO - Fature Finding for Bruker
2020-05-07 21:55:23 INFO - Extracted 261,930 features.
2020-05-07 21:58:25 INFO - First search complete. Compared 356,163,285 spectra and found 1,947,349 psms.
2020-05-07 22:00:18 INFO - Extracted columns for 1,947,349 psms.
2020-05-07 22:00:25 INFO - Scoring peptides complete. For 0.01 FDR found 151,359 targets and 1,030 decoys.
2020-05-07 22:01:11 INFO - Scoring proteins complete. For 0.01 FDR found 149,562 targets and 168 decoys. A total of 6,299 proteins found.
2020-05-07 22




Integration test complete. Time elapsed 3.09 hours
Complete


In [11]:
pd.DataFrame(report, columns = columns)

,convert_raw,create_library,find_features,parallel,ppm,match_between_runs,max_lfq,convert_raw,calibrate,parallel,ppm,n_proteins,n_sequences,n_sequence_charges,time_minutes,error
0,True,True,True,True,True,False,False,True,True,True,True,5884.0,38046.0,43494.0,25.570921,False
1,True,False,True,True,True,False,False,False,True,True,True,5883.0,38043.0,43489.0,18.903161,False
2,True,True,True,True,True,False,False,True,True,True,True,6389.0,55061.0,68114.0,43.418143,False
3,True,False,True,True,True,False,False,False,True,True,True,6389.0,55061.0,68114.0,8.332379,False
4,True,False,False,True,True,False,False,False,True,True,True,6208.0,47703.0,55667.0,2.698864,False
5,True,False,True,True,True,True,False,False,True,True,True,NaN,NaN,NaN,18.673922,True
6,True,False,True,True,True,False,True,False,True,True,True,NaN,NaN,NaN,18.629323,True
7,True,False,True,True,True,False,False,False,False,True,True,5823.0,37991.0,43546.0,17.582534,False
8,True,False,False,True,True,False,False,False,True,True,True,6430.0,57739.0,68990.0,9.181147,False
9,True,False,True,True,True,True,False,False,True,True,True,NaN,NaN,NaN,8.268159,True
